In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Grammar Correction.csv')

In [3]:
df.head()

,Serial Number,Error Type,Ungrammatical Statement,Standard English
0,1,Verb Tense Errors,I goes to the store everyday.,I go to the store everyday.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
2,3,Verb Tense Errors,She have completed her homework.,She has completed her homework.
3,4,Verb Tense Errors,He don't know the answer.,He doesn't know the answer.
4,5,Verb Tense Errors,The sun rise in the east.,The sun rises in the east.


In [4]:
final_df = df.drop(columns=['Error Type','Serial Number'])

In [5]:
final_df.head()

,Ungrammatical Statement,Standard English
0,I goes to the store everyday.,I go to the store everyday.
1,They was playing soccer last night.,They were playing soccer last night.
2,She have completed her homework.,She has completed her homework.
3,He don't know the answer.,He doesn't know the answer.
4,The sun rise in the east.,The sun rises in the east.


In [11]:
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("../Grammar Correction.csv")  # Update with your file path
df = df[["Ungrammatical Statement", "Standard English"]].dropna()

# Rename for convenience
df.columns = ["input", "target"]

# Train/validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["input"].tolist(),
    df["target"].tolist(),
    test_size=0.2,
    random_state=42
)

# Load model and tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenization function
def tokenize_function(input_texts, target_texts):
    inputs = tokenizer(
        ["fix grammar: " + text for text in input_texts],
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    targets = tokenizer(
        target_texts,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return inputs["input_ids"], inputs["attention_mask"], targets["input_ids"]

# Prepare TensorFlow dataset
def create_tf_dataset(inputs, targets):
    input_ids, attention_mask, label_ids = tokenize_function(inputs, targets)
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        },
        label_ids
    ))
    return dataset.shuffle(100).batch(4).prefetch(tf.data.AUTOTUNE)

train_dataset = create_tf_dataset(train_texts, train_labels)
val_dataset = create_tf_dataset(val_texts, val_labels)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# Save the model
model.save_pretrained("./t5_grammar_corrector")
tokenizer.save_pretrained("./t5_grammar_corrector")


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/3

404/404 [==============================] - 424s 986ms/step - loss: 0.8329 - val_loss: 0.2049
Epoch 2/3
404/404 [==============================] - 396s 980ms/step - loss: 0.1897 - val_loss: 0.1243
Epoch 3/3
404/404 [==============================] - 394s 974ms/step - loss: 0.0972 - val_loss: 0.0427


('./t5_grammar_corrector\\tokenizer_config.json',
 './t5_grammar_corrector\\special_tokens_map.json',
 './t5_grammar_corrector\\tokenizer.json')

In [12]:
def correct_grammar(text):
    input_text = "fix grammar: " + text
    inputs = tokenizer(input_text, return_tensors="tf", padding="max_length", truncation=True, max_length=128)
    output = model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example
print(correct_grammar("She go to school every day."))


She goes to school every day.


In [14]:
print(correct_grammar("I goes to school every day"))
print(correct_grammar("He don't like mangoes"))
print(correct_grammar("They was happy with the results"))


I go to school every day.
He doesn't like mangoes.
They were happy with the results.


In [19]:
print(correct_grammar("where you are going"))